In [2]:
import setup
setup.init_django()

In [3]:
from market.models import StockQuote

In [4]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [5]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

12741

In [6]:
total = 0
for obj in qs:
    total += obj.close_price

In [7]:
total / qs.count()

Decimal('370.6656873322345184836355074')

In [8]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 3, 'avg_price': Decimal('634.6686316516709512')}, {'company': 5, 'avg_price': Decimal('428.7963671246006390')}, {'company': 2, 'avg_price': Decimal('197.2389634726688103')}, {'company': 1, 'avg_price': Decimal('233.7022529064620832')}]>

In [9]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [10]:
rolling_qs

[<StockQuote: StockQuote object (2921291)>,
 <StockQuote: StockQuote object (2921292)>,
 <StockQuote: StockQuote object (2921293)>,
 <StockQuote: StockQuote object (2921294)>,
 <StockQuote: StockQuote object (2921295)>]

In [11]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 2921291 234.6501 234.6501
2 2921292 234.2512 234.45065
3 2921293 234.2700 234.3904333333333333333333333
4 2921294 234.2400 234.352825
5 2921295 233.9200 234.26626


In [24]:
latest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date - timedelta(days=40), end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
    .annotate(latest_time=Max('time'))
    .values('company', 'date', 'latest_time')
    .order_by('date')
)

actual_timestamps = [x['latest_time'] for x in latest_daily_timestamps]
actual_timestamps

[datetime.datetime(2024, 11, 25, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 26, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 27, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 28, 0, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 11, 29, 21, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 2, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 3, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 4, 23, 50, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 5, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 6, 23, 50, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 7, 0, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 9, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 10, 23, 55, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 11, 23, 55, tzinfo=datetime.timezone.utc),
 

In [26]:
qs = StockQuote.objects.filter(
    company__ticker=ticker, 
    time__range=(start_date, end_date),
    time__in=actual_timestamps
)

In [27]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [29]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

2921295 233.9200 238.0480600000000000 2025-02-01 00:55:00+00:00
2921283 234.9202 237.2189600000000000 2025-01-31 23:55:00+00:00
2921091 245.0000 234.6689200000000000 2025-01-30 23:55:00+00:00
2925761 238.8001 230.0989200000000000 2025-01-29 23:55:00+00:00
2925569 237.6000 226.9899000000000000 2025-01-28 23:55:00+00:00


In [30]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
                expression=Avg('close_price'),
                order_by=F('time').asc(),
                partition_by=[],
                frame=RowRange(start=-19, end=0),
            )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(2921295,
 Decimal('233.9200'),
 Decimal('238.0480600000000000'),
 Decimal('232.2073300000000000'))